In [ ]:
!pip install snowflake-ml-python --upgrade

# MONAI Lung CT Registration - Distributed Inference

This notebook runs **distributed GPU inference** using Snowflake's `run_batch()` API.

## Key Benefits of `run_batch()`
- **Distributed Processing**: Automatic parallelization across GPU compute pools
- **Scalability**: Process thousands of CT scans concurrently
- **Simplicity**: No Ray/Spark setup required
- **Cost Efficiency**: Auto-scaling compute pools with auto-suspend

## Workflow
1. Load input DataFrame with case IDs and presigned URLs
2. Call `model_version.run_batch()` with GPU compute pool
3. Results saved to output stage

## Requirements
- `snowflake-ml-python >= 1.8.0`
- Model registered with `CustomModel` class (see training notebook)
- GPU compute pool (e.g., `GPU_NV_S` instance family)

## Step 1: Initialize Snowflake Session

Connect to Snowflake to access Model Registry and stages.

In [ ]:
import pandas as pd

from snowflake.snowpark.context import get_active_session
from snowflake.ml.registry import Registry
from snowflake.ml.model import JobSpec, OutputSpec, SaveMode

session = get_active_session()
session.use_database('SF_CLINICAL_DB')
session.use_schema('UTILS')
print("✅ Connected to Snowflake")

## Step 2: Prepare Input Data

Create a Snowpark DataFrame with stage paths for inference. The `run_batch()` method will automatically download and convert these files to bytes using `InputSpec`.

The DataFrame will have these columns:
- `CASE_ID`: Unique identifier for each case
- `FIXED_PATH`: Stage path to expiration CT scan
- `MOVING_PATH`: Stage path to inspiration CT scan  
- `FIXED_LABEL_PATH`: Stage path to expiration lung mask
- `MOVING_LABEL_PATH`: Stage path to inspiration lung mask

In [ ]:
def load_inference_cases(stage_location="@SF_CLINICAL_DB.UTILS.MONAI_MEDICAL_IMAGES_STG", target_count=None):
    """
    Load paired CT scan paths for inference.
    If target_count is specified, duplicate cases to reach that count for performance testing.
    """
    df_files = session.sql(
        f"LIST {stage_location} PATTERN = '.*.nii.gz'"
    ).select('"name"').to_pandas()
    
    df_files["name"] = "UTILS." + df_files["name"]
    
    fixed_imgs = df_files[df_files['name'].str.contains("scans.*_exp", regex=True)]
    moving_imgs = df_files[df_files['name'].str.contains("scans.*_insp", regex=True)]
    fixed_masks = df_files[df_files['name'].str.contains("lungMasks.*_exp", regex=True)]
    moving_masks = df_files[df_files['name'].str.contains("lungMasks.*_insp", regex=True)]
    
    pairs = []
    for _, row in fixed_imgs.iterrows():
        f_path = row['name']
        filename = f_path.split('/')[-1]
        case_id = filename.split('_exp')[0]
        
        m_path = moving_imgs[moving_imgs['name'].str.contains(f"{case_id}_insp")].iloc[0]['name']
        fl_path = fixed_masks[fixed_masks['name'].str.contains(f"{case_id}_exp")].iloc[0]['name']
        ml_path = moving_masks[moving_masks['name'].str.contains(f"{case_id}_insp")].iloc[0]['name']
        
        pairs.append({
            "CASE_ID": case_id,
            "FIXED_PATH": f"@SF_CLINICAL_DB.{f_path}",
            "MOVING_PATH": f"@SF_CLINICAL_DB.{m_path}",
            "FIXED_LABEL_PATH": f"@SF_CLINICAL_DB.{fl_path}",
            "MOVING_LABEL_PATH": f"@SF_CLINICAL_DB.{ml_path}"
        })
    
    print(f"Found {len(pairs)} unique cases")
    
    # Duplicate cases for performance testing
    if target_count and target_count > len(pairs):
        original_pairs = pairs.copy()
        idx = 0
        while len(pairs) < target_count:
            dup = original_pairs[idx % len(original_pairs)].copy()
            dup["CASE_ID"] = f"{dup['CASE_ID']}_dup{len(pairs)}"  # Unique ID for duplicates
            pairs.append(dup)
            idx += 1
        print(f"Duplicated to {len(pairs)} cases for performance testing")
    
    return session.create_dataframe(pd.DataFrame(pairs))

# Load cases and duplicate to 1000 for performance testing
input_df = load_inference_cases(target_count=1000)
print(f"\nTotal cases for inference: {input_df.count()}")
input_df.limit(100).to_pandas()

## Step 3: Configure GPU Compute Pool

The `run_batch()` method requires a GPU compute pool. Verify the pool exists and set the pool name:

In [ ]:
SHOW COMPUTE POOLS;

In [ ]:
COMPUTE_POOL='GPU_ML_M_POOL'

## Step 4: Run Distributed Batch Inference

Now we use `run_batch()` to process all cases on the GPU compute pool.

**Key Benefits:**
- Snowflake automatically builds container images with dependencies
- Ray orchestration is handled internally
- Results are saved to the specified output stage
- Job progress is visible in Snowsight UI

In [ ]:
# ============================================================================
# RUN DISTRIBUTED BATCH INFERENCE
# ============================================================================

from snowflake.ml.model import InputSpec, InputFormat, FileEncoding

registry = Registry(
    session=session,
    database_name="SF_CLINICAL_DB",
    schema_name="UTILS"
)

MODEL_NAME = "LUNG_CT_REGISTRATION"
MODEL_VERSION = "V_RUN_BATCH_8"

mv = registry.get_model(MODEL_NAME).version(MODEL_VERSION)
print(f"Loaded model: {mv.fully_qualified_model_name}")

print(f"\nStarting batch inference on {COMPUTE_POOL}...")
print(f"Cases: {input_df.count()}")

job = mv.run_batch(
    compute_pool=COMPUTE_POOL,
    X=input_df,
    input_spec=InputSpec(
        column_handling={
            "FIXED_PATH": {"input_format": InputFormat.FULL_STAGE_PATH, "convert_to": FileEncoding.RAW_BYTES},
            "MOVING_PATH": {"input_format": InputFormat.FULL_STAGE_PATH, "convert_to": FileEncoding.RAW_BYTES},
            "FIXED_LABEL_PATH": {"input_format": InputFormat.FULL_STAGE_PATH, "convert_to": FileEncoding.RAW_BYTES},
            "MOVING_LABEL_PATH": {"input_format": InputFormat.FULL_STAGE_PATH, "convert_to": FileEncoding.RAW_BYTES}
        }
    ),
    output_spec=OutputSpec(
        stage_location="@SF_CLINICAL_DB.UTILS.RESULTS_STG/inference_out/",
        mode=SaveMode.OVERWRITE
    ),
    job_spec=JobSpec(gpu_requests="1")
)

print(f"\nJob submitted! ID: {job.id}")
print(f"Monitor in Snowsight: Jobs & Services")

## Step 5: Monitor Job Progress

Check the job status and wait for completion.

In [ ]:
# Check job status
print(f"Job Status: {job.status}")

job.wait()

print("Job completed!")
job.show_logs()

## Step 6: Review Results

The inference results are saved to the output stage and returned as a DataFrame.

In [ ]:
ls @SF_CLINICAL_DB.UTILS.RESULTS_STG/inference_out

In [ ]:

select $1 as res
from @SF_CLINICAL_DB.UTILS.results_stg/inference_out
 (file_format => sf_clinical_db.utils.parquet_ff, PATTERN => '.*\.parquet') 

In [ ]:
import json

res_df = session.sql(r"""
SELECT $1 AS res
FROM @SF_CLINICAL_DB.UTILS.RESULTS_STG/inference_out
(FILE_FORMAT => SF_CLINICAL_DB.UTILS.PARQUET_FF, PATTERN => '.*\.parquet') 
""").to_pandas()

# Parse JSON and extract key metrics
results = [json.loads(row) for row in res_df['RES']]
results_df = pd.DataFrame(results)

# Display clean summary
print("📊 Inference Results")
print("=" * 50)
display_df = results_df[['CASE_ID', 'STATUS', 'DICE_SCORE', 'DDF_MAGNITUDE', 'DEVICE_USED']].copy()
display_df['DICE_SCORE'] = display_df['DICE_SCORE'].round(4)
display_df['DDF_MAGNITUDE'] = display_df['DDF_MAGNITUDE'].round(4)
display(display_df)

# Summary statistics
successful = results_df[results_df['STATUS'] == 'success']
print(f"\n📈 Summary:")
print(f"   Total cases: {len(results_df)}")
print(f"   Successful: {len(successful)}")
print(f"   Failed: {len(results_df) - len(successful)}")
if len(successful) > 0:
    print(f"   Mean Dice: {successful['DICE_SCORE'].mean():.4f}")
    print(f"   Min Dice: {successful['DICE_SCORE'].min():.4f}")
    print(f"   Max Dice: {successful['DICE_SCORE'].max():.4f}")
    print(f"   Mean DDF Magnitude: {successful['DDF_MAGNITUDE'].mean():.4f}")

In [ ]:
# Display inference results - clean view
import json

print("📊 Inference Results:")
print("=" * 70)

# Parse JSON results from parquet
results_list = [json.loads(row['RES']) for _, row in res_df.iterrows()]
results_df = pd.DataFrame(results_list)

# Select only the key columns for display
display_cols = ['CASE_ID', 'STATUS', 'DICE_SCORE', 'DDF_MAGNITUDE', 'DEVICE_USED']
print(results_df[display_cols].to_string(index=False))

# Summary statistics
print("\n" + "=" * 70)
print("📈 Summary Statistics:")
print("=" * 70)

successful = results_df[results_df['STATUS'] == 'success']
failed = results_df[results_df['STATUS'] != 'success']

print(f"   Total cases:    {len(results_df)}")
print(f"   Successful:     {len(successful)}")
print(f"   Failed:         {len(failed)}")

if len(successful) > 0:
    print(f"\n   Dice Score:")
    print(f"      Mean:        {successful['DICE_SCORE'].mean():.4f}")
    print(f"      Std:         {successful['DICE_SCORE'].std():.4f}")
    print(f"      Min:         {successful['DICE_SCORE'].min():.4f}")
    print(f"      Max:         {successful['DICE_SCORE'].max():.4f}")
    
    print(f"\n   DDF Magnitude:")
    print(f"      Mean:        {successful['DDF_MAGNITUDE'].mean():.4f}")
    print(f"      Min:         {successful['DDF_MAGNITUDE'].min():.4f}")
    print(f"      Max:         {successful['DDF_MAGNITUDE'].max():.4f}")
    
    print(f"\n   Device:         {successful['DEVICE_USED'].iloc[0]}")

if len(failed) > 0:
    print(f"\n⚠️  Failed cases:")
    for _, row in failed.iterrows():
        print(f"      {row['CASE_ID']}: {row['STATUS']}")

## Step 7: Save Results to Table (Optional)

Persist the inference results to a Snowflake table for querying.

In [ ]:
# Save results to Snowflake table with parsed columns
RESULTS_TABLE = "SF_CLINICAL_DB.UTILS.MONAI_INFERENCE_RESULTS"

session.sql(f"""
CREATE OR REPLACE TABLE {RESULTS_TABLE} AS
SELECT 
    $1:CASE_ID::VARCHAR AS CASE_ID,
    $1:STATUS::VARCHAR AS STATUS,
    $1:DICE_SCORE::FLOAT AS DICE_SCORE,
    $1:DDF_MAGNITUDE::FLOAT AS DDF_MAGNITUDE,
    $1:DEVICE_USED::VARCHAR AS DEVICE_USED,
    CURRENT_TIMESTAMP() AS INFERENCE_TIMESTAMP
FROM @SF_CLINICAL_DB.UTILS.RESULTS_STG/inference_out
(FILE_FORMAT => SF_CLINICAL_DB.UTILS.PARQUET_FF, PATTERN => '.*\\.parquet')
""").collect()

print(f"✅ Results saved to {RESULTS_TABLE}")

# Display saved results
session.table(RESULTS_TABLE).to_pandas()

## Summary

🎉 **Inference completed using `run_batch()`!**

### What We Did
1. ✅ Loaded input DataFrame with case paths
2. ✅ Called `model.run_batch()` on GPU compute pool
3. ✅ Processed all cases in parallel
4. ✅ Saved results to stage and table

### Output Locations
- **Stage**: `@SF_CLINICAL_DB.UTILS.RESULTS_STG/run_batch_output/`
- **Table**: `SF_CLINICAL_DB.RESULTS.MONAI_INFERENCE_RESULTS`

### Advantages Over Ray map_batches
- **Simpler code**: ~50 lines vs ~400 lines
- **No manual dependency management**: Snowflake builds container images
- **Better monitoring**: Jobs visible in Snowsight UI
- **Automatic cleanup**: No need to manage Ray cluster lifecycle